In [1]:
import os
import numpy as np
import pandas as pd
import torch
import os
import json
import pandas as pd
from PIL import Image
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import cv2
import numpy as np
%matplotlib inline
from tqdm import tqdm
import glob
import pickle

In [2]:
DATA_DIR = '/mnt/chicm/data/open-images/relation'
DETECT_DATA_DIR = '/mnt/chicm/data/open-images/detect'

In [3]:
!ls sub_detect_57 -lh

total 2.8G
-rw-rw-r-- 1 core core 242M Aug 19 15:51 sub_cas_800_detect_57_0819_lb06005.csv
-rw-rw-r-- 1 core core 266M Jul 22 16:57 sub_detect_0722_1.csv
-rw-rw-r-- 1 core core 252M Jul 24 17:01 sub_detect_0724_lb05524.csv
-rw-rw-r-- 1 core core 233M Jul 30 08:18 sub_detect_57_0730_lb05849.csv
-rw-rw-r-- 1 core core 259M Aug  5 07:41 sub_detect_57_cas_1024_0805_1_lb06012.csv
-rw-rw-r-- 1 core core 363M Aug 20 14:25 sub_detect_57_cas_1024_top100.csv
-rw-rw-r-- 1 core core 396M Aug 20 14:19 sub_detect_57_cas_800_top100.csv
-rw-rw-r-- 1 core core 240M Aug  4 13:36 sub_detect_cas_1024_57_0804_lb06067.csv
-rw-rw-r-- 1 core core 286M Aug 18 17:43 sub_retina_detect_57_0818_lb04842.csv
-rw-rw-r-- 1 core core 286M Aug  9 18:00 sub_retina_x101_detect_0810_lb03783.csv


In [94]:
df1 = pd.read_csv('sub_detect_57/sub_detect_57_cas_1024_top100.csv')
df2 = pd.read_csv('sub_detect_57/sub_detect_57_cas_800_top100.csv')
df3 = pd.read_csv('sub_detect_57/sub_detect_57_0730_lb05849.csv')
df1.head()

,ImageId,PredictionString
0,21135d5ed0e7a0b7,/m/04yx4 0.8413025 0.3771222 0.2306115 0.74984...
1,763c72626bf24534,/m/04yx4 0.9781426 0.0050735 0.1911065 0.24955...
2,7540c162180973b1,/m/04yx4 0.7035857 0.7911549 0.1323342 0.85766...
3,c4e522dce52cb02f,/m/04yx4 0.9637073 0.3919787 0.2624895 0.54916...
4,cafcd9496c808085,/m/0k4j 0.0035383 0.7955955 0.0142576 0.999023...


In [39]:
# lb 0.06179
#df1 = pd.read_csv('sub_detect_57/sub_detect_cas_1024_57_0804_lb06067.csv')
#df2 = pd.read_csv('sub_detect_57/sub_detect_57_0730_lb05849.csv')
#df3 = pd.read_csv('sub_detect_57/sub_detect_0724_lb05524.csv')
#df1.head()

,ImageId,PredictionString
0,21135d5ed0e7a0b7,/m/04yx4 0.8447121 0.3793392 0.2354858 0.75110...
1,763c72626bf24534,/m/04yx4 0.9533325 0.0052673 0.1912388 0.25071...
2,7540c162180973b1,/m/04yx4 0.6267715 0.7935570 0.1335864 0.85723...
3,c4e522dce52cb02f,/m/04yx4 0.8973290 0.3921414 0.2635090 0.54854...
4,cafcd9496c808085,/m/0k4j 0.0023795 0.7963226 0.0179788 0.997849...


In [72]:
df1 = pd.read_csv('sub_detect_57/sub_detect_cas_1024_57_0804_lb06067.csv')
df2 = pd.read_csv('sub_detect_57/sub_cas_800_detect_57_0819_lb06005.csv')
df3 = pd.read_csv('sub_detect_57/sub_detect_57_0730_lb05849.csv')
df1.head()

,ImageId,PredictionString
0,21135d5ed0e7a0b7,/m/04yx4 0.8447121 0.3793392 0.2354858 0.75110...
1,763c72626bf24534,/m/04yx4 0.9533325 0.0052673 0.1912388 0.25071...
2,7540c162180973b1,/m/04yx4 0.6267715 0.7935570 0.1335864 0.85723...
3,c4e522dce52cb02f,/m/04yx4 0.8973290 0.3921414 0.2635090 0.54854...
4,cafcd9496c808085,/m/0k4j 0.0023795 0.7963226 0.0179788 0.997849...


In [58]:
#ImageID,PredictionString
#ImageID,{Label Confidence XMin YMin XMax YMax} {...}

In [95]:
def get_dets_by_pred_str(pred_str):
    try:
        if len(pred_str) < 1:
            return []
    except:
        print('pred_str:', pred_str)
        raise
    det_items = pred_str.split(' ')
    dets, det = [], []
    for i in range(len(det_items)):
        det.append(det_items[i])
        if (i+1) % 6 == 0:
            dets.append(det)
            det = []
    return dets

In [96]:
df1['dets'] = df1.PredictionString.fillna('').map(lambda x: get_dets_by_pred_str(x))
df2['dets'] = df2.PredictionString.fillna('').map(lambda x: get_dets_by_pred_str(x))
df3['dets'] = df3.PredictionString.fillna('').map(lambda x: get_dets_by_pred_str(x))
df1.head()

,ImageId,PredictionString,dets
0,21135d5ed0e7a0b7,/m/04yx4 0.8413025 0.3771222 0.2306115 0.74984...,"[[/m/04yx4, 0.8413025, 0.3771222, 0.2306115, 0..."
1,763c72626bf24534,/m/04yx4 0.9781426 0.0050735 0.1911065 0.24955...,"[[/m/04yx4, 0.9781426, 0.0050735, 0.1911065, 0..."
2,7540c162180973b1,/m/04yx4 0.7035857 0.7911549 0.1323342 0.85766...,"[[/m/04yx4, 0.7035857, 0.7911549, 0.1323342, 0..."
3,c4e522dce52cb02f,/m/04yx4 0.9637073 0.3919787 0.2624895 0.54916...,"[[/m/04yx4, 0.9637073, 0.3919787, 0.2624895, 0..."
4,cafcd9496c808085,/m/0k4j 0.0035383 0.7955955 0.0142576 0.999023...,"[[/m/0k4j, 0.0035383, 0.7955955, 0.0142576, 0...."


In [97]:
len(df1.iloc[0].dets)

100

In [98]:
def dets_to_xywh(dets):
    res = []
    for det in dets:
        label = det[0]
        conf = float(det[1])
        xmin = float(det[2])
        ymin = float(det[3])
        xmax = float(det[4])
        ymax = float(det[5])
        new_det = [(xmin+xmax)/2, (ymin+ymax)/2, xmax-xmin, ymax-ymin, label, conf]
        res.append(new_det)
    return res                                                                 

In [99]:
dets1 = df1.dets.map(lambda x: dets_to_xywh(x))
dets2 = df2.dets.map(lambda x: dets_to_xywh(x))
dets3 = df3.dets.map(lambda x: dets_to_xywh(x))

In [100]:
len(dets3)

99999

In [102]:
final_ens = []
for i in tqdm(range(len(dets1))):
    #ens = GeneralEnsemble([dets1[i], dets2[i], dets3[i]], weights=[0.5, 0.3, 0.2])
    ens = GeneralEnsemble([dets1[i], dets2[i], dets3[i]], weights=[0.5, 0.3, 0.2])
    final_ens.append(ens)



  0%|          | 0/99999 [00:00<?, ?it/s]

  0%|          | 2/99999 [00:00<1:23:25, 19.98it/s]

  0%|          | 4/99999 [00:00<1:23:26, 19.97it/s]

  0%|          | 9/99999 [00:00<1:09:47, 23.88it/s]

  0%|          | 13/99999 [00:00<1:03:54, 26.08it/s]

  0%|          | 16/99999 [00:00<1:02:51, 26.51it/s]

  0%|          | 19/99999 [00:00<1:11:31, 23.30it/s]

  0%|          | 22/99999 [00:00<1:09:39, 23.92it/s]

  0%|          | 25/99999 [00:00<1:08:51, 24.20it/s]

  0%|          | 28/99999 [00:01<1:09:42, 23.90it/s]

  0%|          | 32/99999 [00:01<1:03:04, 26.41it/s]

  0%|          | 35/99999 [00:01<1:02:19, 26.73it/s]

  0%|          | 40/99999 [00:01<53:50, 30.95it/s]  

  0%|          | 44/99999 [00:01<58:25, 28.52it/s]

  0%|          | 48/99999 [00:01<1:02:10, 26.79it/s]

  0%|          | 53/99999 [00:01<54:58, 30.30it/s]  

  0%|          | 57/99999 [00:02<57:27, 28.99it/s]

  0%|          | 62/99999 [00:02<53:05, 31.37it/s]

  0%|          | 66/99999 [00:02<1:01:05, 27.2

  1%|          | 1196/99999 [00:37<1:07:40, 24.33it/s]

  1%|          | 1201/99999 [00:37<57:44, 28.51it/s]  

  1%|          | 1205/99999 [00:37<1:01:35, 26.73it/s]

  1%|          | 1210/99999 [00:37<53:17, 30.89it/s]  

  1%|          | 1215/99999 [00:38<48:55, 33.65it/s]

  1%|          | 1219/99999 [00:38<1:00:38, 27.15it/s]

  1%|          | 1224/99999 [00:38<56:11, 29.30it/s]  

  1%|          | 1228/99999 [00:38<58:32, 28.12it/s]

  1%|          | 1232/99999 [00:38<59:04, 27.87it/s]

  1%|          | 1235/99999 [00:38<59:51, 27.50it/s]

  1%|          | 1239/99999 [00:38<55:03, 29.90it/s]

  1%|          | 1243/99999 [00:39<59:45, 27.54it/s]

  1%|          | 1246/99999 [00:39<1:10:17, 23.41it/s]

  1%|          | 1249/99999 [00:39<1:14:13, 22.18it/s]

  1%|▏         | 1252/99999 [00:39<1:18:58, 20.84it/s]

  1%|▏         | 1255/99999 [00:39<1:17:21, 21.28it/s]

  1%|▏         | 1259/99999 [00:39<1:09:11, 23.78it/s]

  1%|▏         | 1262/99999 [00:40<1:15:12, 21.88it/s]

  1%

  2%|▏         | 2384/99999 [01:14<49:45, 32.70it/s]

  2%|▏         | 2388/99999 [01:14<55:29, 29.31it/s]

  2%|▏         | 2392/99999 [01:15<58:52, 27.63it/s]

  2%|▏         | 2396/99999 [01:15<58:48, 27.66it/s]

  2%|▏         | 2401/99999 [01:15<53:10, 30.59it/s]

  2%|▏         | 2406/99999 [01:15<51:10, 31.79it/s]

  2%|▏         | 2410/99999 [01:15<54:22, 29.91it/s]

  2%|▏         | 2414/99999 [01:15<51:57, 31.30it/s]

  2%|▏         | 2418/99999 [01:15<58:13, 27.93it/s]

  2%|▏         | 2425/99999 [01:16<49:27, 32.88it/s]

  2%|▏         | 2430/99999 [01:16<45:26, 35.78it/s]

  2%|▏         | 2434/99999 [01:16<47:39, 34.12it/s]

  2%|▏         | 2440/99999 [01:16<44:25, 36.60it/s]

  2%|▏         | 2444/99999 [01:16<44:21, 36.65it/s]

  2%|▏         | 2449/99999 [01:16<45:20, 35.85it/s]

  2%|▏         | 2453/99999 [01:16<54:35, 29.78it/s]

  2%|▏         | 2457/99999 [01:17<1:01:27, 26.45it/s]

  2%|▏         | 2461/99999 [01:17<58:40, 27.71it/s]  

  2%|▏         | 2464/99

  4%|▎         | 3605/99999 [01:52<49:46, 32.27it/s]

  4%|▎         | 3611/99999 [01:52<45:55, 34.98it/s]

  4%|▎         | 3622/99999 [01:52<38:17, 41.95it/s]

  4%|▎         | 3628/99999 [01:52<38:48, 41.39it/s]

  4%|▎         | 3634/99999 [01:52<35:34, 45.15it/s]

  4%|▎         | 3640/99999 [01:53<48:23, 33.19it/s]

  4%|▎         | 3647/99999 [01:53<41:20, 38.84it/s]

  4%|▎         | 3652/99999 [01:53<41:09, 39.01it/s]

  4%|▎         | 3657/99999 [01:53<39:50, 40.30it/s]

  4%|▎         | 3662/99999 [01:53<40:21, 39.79it/s]

  4%|▎         | 3670/99999 [01:53<34:54, 45.99it/s]

  4%|▎         | 3676/99999 [01:54<39:13, 40.92it/s]

  4%|▎         | 3684/99999 [01:54<33:54, 47.35it/s]

  4%|▎         | 3690/99999 [01:54<36:05, 44.47it/s]

  4%|▎         | 3697/99999 [01:54<34:42, 46.25it/s]

  4%|▎         | 3703/99999 [01:54<33:48, 47.46it/s]

  4%|▎         | 3709/99999 [01:54<35:27, 45.26it/s]

  4%|▎         | 3714/99999 [01:54<39:16, 40.86it/s]

  4%|▎         | 3719/99999 

  5%|▍         | 4975/99999 [02:30<1:05:10, 24.30it/s]

  5%|▍         | 4978/99999 [02:30<1:02:08, 25.49it/s]

  5%|▍         | 4981/99999 [02:31<1:03:08, 25.08it/s]

  5%|▍         | 4985/99999 [02:31<1:01:34, 25.72it/s]

  5%|▍         | 4990/99999 [02:31<54:32, 29.04it/s]  

  5%|▍         | 4994/99999 [02:31<58:59, 26.84it/s]

  5%|▍         | 4997/99999 [02:31<1:02:17, 25.42it/s]

  5%|▌         | 5000/99999 [02:31<1:07:07, 23.59it/s]

  5%|▌         | 5004/99999 [02:31<1:01:55, 25.57it/s]

  5%|▌         | 5008/99999 [02:31<57:16, 27.64it/s]  

  5%|▌         | 5011/99999 [02:32<1:00:37, 26.11it/s]

  5%|▌         | 5016/99999 [02:32<52:56, 29.90it/s]  

  5%|▌         | 5020/99999 [02:32<1:03:22, 24.98it/s]

  5%|▌         | 5024/99999 [02:32<59:44, 26.50it/s]  

  5%|▌         | 5031/99999 [02:32<49:42, 31.85it/s]

  5%|▌         | 5035/99999 [02:32<47:30, 33.31it/s]

  5%|▌         | 5039/99999 [02:32<52:51, 29.94it/s]

  5%|▌         | 5043/99999 [02:33<52:27, 30.17it/s]

  

...

 94%|█████████▍| 93927/99999 [47:14<02:54, 34.70it/s]

 94%|█████████▍| 93937/99999 [47:14<02:21, 42.72it/s]

 94%|█████████▍| 93943/99999 [47:14<02:16, 44.31it/s]

 94%|█████████▍| 93949/99999 [47:15<02:34, 39.06it/s]

 94%|█████████▍| 93959/99999 [47:15<02:10, 46.26it/s]

 94%|█████████▍| 93965/99999 [47:15<02:21, 42.72it/s]

 94%|█████████▍| 93971/99999 [47:15<02:29, 40.21it/s]

 94%|█████████▍| 93978/99999 [47:15<02:20, 42.86it/s]

 94%|█████████▍| 93983/99999 [47:15<02:47, 35.98it/s]

 94%|█████████▍| 93994/99999 [47:15<02:13, 45.08it/s]

 94%|█████████▍| 94001/99999 [47:16<02:22, 42.08it/s]

 94%|█████████▍| 94010/99999 [47:16<02:07, 46.82it/s]

 94%|█████████▍| 94016/99999 [47:16<02:15, 44.21it/s]

 94%|█████████▍| 94022/99999 [47:16<02:16, 43.73it/s]

 94%|█████████▍| 94027/99999 [47:16<02:25, 41.16it/s]

 94%|█████████▍| 94032/99999 [47:16<02:48, 35.40it/s]

 94%|█████████▍| 94041/99999 [47:17<02:20, 42.52it/s]

 94%|█████████▍| 94047/99999 [47:17<02:27, 40.26it/s]

 94%|█████

 95%|█████████▌| 95284/99999 [47:52<02:17, 34.32it/s]

 95%|█████████▌| 95288/99999 [47:52<02:22, 33.02it/s]

 95%|█████████▌| 95292/99999 [47:53<02:23, 32.70it/s]

 95%|█████████▌| 95297/99999 [47:53<02:10, 36.13it/s]

 95%|█████████▌| 95301/99999 [47:53<02:14, 35.01it/s]

 95%|█████████▌| 95305/99999 [47:53<02:25, 32.16it/s]

 95%|█████████▌| 95309/99999 [47:53<02:50, 27.48it/s]

 95%|█████████▌| 95312/99999 [47:53<03:06, 25.09it/s]

 95%|█████████▌| 95316/99999 [47:53<02:46, 28.07it/s]

 95%|█████████▌| 95320/99999 [47:54<02:34, 30.28it/s]

 95%|█████████▌| 95324/99999 [47:54<02:48, 27.82it/s]

 95%|█████████▌| 95327/99999 [47:54<02:57, 26.38it/s]

 95%|█████████▌| 95330/99999 [47:54<02:57, 26.36it/s]

 95%|█████████▌| 95333/99999 [47:54<03:19, 23.45it/s]

 95%|█████████▌| 95336/99999 [47:54<03:17, 23.62it/s]

 95%|█████████▌| 95340/99999 [47:54<03:03, 25.37it/s]

 95%|█████████▌| 95343/99999 [47:54<02:56, 26.42it/s]

 95%|█████████▌| 95346/99999 [47:55<02:57, 26.23it/s]

 95%|█████

 97%|█████████▋| 96512/99999 [48:30<02:06, 27.49it/s]

 97%|█████████▋| 96518/99999 [48:30<01:48, 31.98it/s]

 97%|█████████▋| 96522/99999 [48:30<01:57, 29.55it/s]

 97%|█████████▋| 96526/99999 [48:30<02:10, 26.70it/s]

 97%|█████████▋| 96531/99999 [48:30<01:52, 30.73it/s]

 97%|█████████▋| 96535/99999 [48:30<02:09, 26.69it/s]

 97%|█████████▋| 96539/99999 [48:31<02:04, 27.82it/s]

 97%|█████████▋| 96543/99999 [48:31<02:16, 25.39it/s]

 97%|█████████▋| 96547/99999 [48:31<02:11, 26.29it/s]

 97%|█████████▋| 96554/99999 [48:31<01:51, 30.99it/s]

 97%|█████████▋| 96558/99999 [48:31<02:00, 28.46it/s]

 97%|█████████▋| 96562/99999 [48:31<01:51, 30.76it/s]

 97%|█████████▋| 96566/99999 [48:31<01:59, 28.64it/s]

 97%|█████████▋| 96570/99999 [48:32<01:56, 29.35it/s]

 97%|█████████▋| 96575/99999 [48:32<01:50, 31.04it/s]

 97%|█████████▋| 96579/99999 [48:32<02:01, 28.12it/s]

 97%|█████████▋| 96582/99999 [48:32<02:04, 27.36it/s]

 97%|█████████▋| 96585/99999 [48:32<02:02, 27.90it/s]

 97%|█████

 98%|█████████▊| 97917/99999 [49:08<00:45, 45.69it/s]

 98%|█████████▊| 97923/99999 [49:09<00:42, 48.56it/s]

 98%|█████████▊| 97929/99999 [49:09<00:44, 46.63it/s]

 98%|█████████▊| 97935/99999 [49:09<00:44, 46.65it/s]

 98%|█████████▊| 97943/99999 [49:09<00:40, 50.65it/s]

 98%|█████████▊| 97950/99999 [49:09<00:37, 54.46it/s]

 98%|█████████▊| 97959/99999 [49:09<00:34, 59.47it/s]

 98%|█████████▊| 97966/99999 [49:09<00:36, 55.92it/s]

 98%|█████████▊| 97972/99999 [49:10<00:50, 40.13it/s]

 98%|█████████▊| 97979/99999 [49:10<00:45, 43.92it/s]

 98%|█████████▊| 97985/99999 [49:10<00:54, 37.20it/s]

 98%|█████████▊| 97990/99999 [49:10<01:00, 33.17it/s]

 98%|█████████▊| 97994/99999 [49:10<00:59, 33.64it/s]

 98%|█████████▊| 98001/99999 [49:10<00:50, 39.44it/s]

 98%|█████████▊| 98006/99999 [49:10<00:56, 35.55it/s]

 98%|█████████▊| 98011/99999 [49:11<01:01, 32.14it/s]

 98%|█████████▊| 98015/99999 [49:11<01:07, 29.30it/s]

 98%|█████████▊| 98019/99999 [49:11<01:13, 26.97it/s]

 98%|█████

 99%|█████████▉| 99220/99999 [49:46<00:29, 26.34it/s]

 99%|█████████▉| 99223/99999 [49:46<00:30, 25.20it/s]

 99%|█████████▉| 99226/99999 [49:46<00:31, 24.84it/s]

 99%|█████████▉| 99229/99999 [49:47<00:29, 26.01it/s]

 99%|█████████▉| 99233/99999 [49:47<00:26, 28.92it/s]

 99%|█████████▉| 99237/99999 [49:47<00:26, 28.92it/s]

 99%|█████████▉| 99241/99999 [49:47<00:26, 28.96it/s]

 99%|█████████▉| 99245/99999 [49:47<00:24, 30.68it/s]

 99%|█████████▉| 99249/99999 [49:47<00:25, 29.55it/s]

 99%|█████████▉| 99253/99999 [49:47<00:26, 28.66it/s]

 99%|█████████▉| 99256/99999 [49:47<00:27, 26.78it/s]

 99%|█████████▉| 99259/99999 [49:48<00:28, 25.82it/s]

 99%|█████████▉| 99265/99999 [49:48<00:24, 30.12it/s]

 99%|█████████▉| 99269/99999 [49:48<00:25, 28.84it/s]

 99%|█████████▉| 99273/99999 [49:48<00:24, 29.23it/s]

 99%|█████████▉| 99277/99999 [49:48<00:23, 31.12it/s]

 99%|█████████▉| 99282/99999 [49:48<00:20, 34.47it/s]

 99%|█████████▉| 99289/99999 [49:48<00:17, 40.37it/s]

 99%|█████

In [80]:
final_ens[0]

[[0.56377102,
  0.616440945,
  0.37436073,
  0.7535090000000001,
  '/m/04yx4',
  0.821652635],
 [0.2019733275,
  0.5523209775,
  0.39575344500000004,
  0.8450368149999999,
  '/m/04yx4',
  0.801308715],
 [0.5500065399999999,
  0.40114056750000004,
  0.33289649,
  0.615221865,
  '/m/04yx4',
  0.35381814500000003],
 [0.7477894374999999,
  0.4853646575,
  0.175953245,
  0.658904945,
  '/m/04yx4',
  0.454613025],
 [0.605087845,
  0.31970566499999997,
  0.22155781999999996,
  0.45964407000000007,
  '/m/04yx4',
  0.26499747500000004],
 [0.483416635, 0.577116695, 0.77806571, 0.84034806, '/m/04yx4', 0.227554135],
 [0.6128938425000001,
  0.19094582000000002,
  0.18455188499999997,
  0.19637355,
  '/m/04yx4',
  0.356437895],
 [0.8984813274999999,
  0.5366655549999999,
  0.19892873500000002,
  0.7729406,
  '/m/04yx4',
  0.268657645],
 [0.6710725384615385,
  0.45056303846153845,
  0.2929644769230769,
  0.6438377384615384,
  '/m/04yx4',
  0.053706725000000004],
 [0.915158155,
  0.33678420249999996,


In [26]:
top_50_ens = []
for i in tqdm(range(len(final_ens))):
    dets = sorted(final_ens[i], key=lambda x: x[5], reverse=True)[:50]
    top_50_ens.append(dets)


100%|██████████| 99999/99999 [00:02<00:00, 45083.93it/s]

In [81]:
len(top_50_ens[0])

50

In [103]:
def xywh_to_pre_str(dets):
    res = []
    if len(dets) < 1:
        return ''
    for det in dets:
        label = det[4]
        conf = det[5]
        xmin = det[0] - (det[2] / 2)
        xmax = det[0] + (det[2] / 2)
        ymin = det[1] - (det[3] / 2)
        ymax = det[1] + (det[3] / 2)
        
        res.append(label)
        res.append('{:.7f}'.format(conf))
        res.append('{:.7f}'.format(xmin))
        res.append('{:.7f}'.format(ymin))
        res.append('{:.7f}'.format(xmax))
        res.append('{:.7f}'.format(ymax))

    res = [str(x) for x in res]
    return ' '.join(res)

In [104]:
pred_strings = pd.Series(final_ens).map(lambda x: xywh_to_pre_str(x)).values
len(pred_strings)

99999

In [86]:
#pred_strings_top50 = pd.Series(top_50_ens).map(lambda x: xywh_to_pre_str(x)).values
#len(pred_strings_top50)

In [105]:
df_sub = pd.read_csv(os.path.join(DATA_DIR, 'VRD_sample_submission.csv'))
df_sub.PredictionString = pred_strings
df_sub.head()

,ImageId,PredictionString
0,21135d5ed0e7a0b7,/m/04yx4 0.8209846 0.3756532 0.2369880 0.75013...
1,763c72626bf24534,/m/04yx4 0.9761059 0.0059799 0.1897154 0.24897...
2,7540c162180973b1,/m/04yx4 0.6516410 0.7939431 0.1332737 0.85725...
3,c4e522dce52cb02f,/m/04yx4 0.9427195 0.3922202 0.2638703 0.54933...
4,cafcd9496c808085,/m/0k4j 0.0011794 0.7955955 0.0142576 0.999023...


In [52]:
#df_sub.to_csv('ensemble_detect_57_0808_2_with_weights.csv', columns=['ImageId', 'PredictionString'], index=False)

In [88]:
#df_sub.to_csv('ensemble_detect_57_0819_1.csv', columns=['ImageId', 'PredictionString'], index=False)

#df1 = pd.read_csv('sub_detect_57/sub_detect_cas_1024_57_0804_lb06067.csv')
#df2 = pd.read_csv('sub_detect_57/sub_cas_800_detect_57_0819_lb06005.csv')
#df3 = pd.read_csv('sub_detect_57/sub_retina_detect_57_0818_lb04842.csv')
# weights: 0.5, 0.4, 0.1

In [106]:
df_sub.to_csv('ensemble_detect_57_0820_1_top100.csv', columns=['ImageId', 'PredictionString'], index=False)

#df1 = pd.read_csv('sub_detect_57/sub_detect_cas_1024_57_0804_lb06067.csv')
#df2 = pd.read_csv('sub_detect_57/sub_cas_800_detect_57_0819_lb06005.csv')
#df3 = pd.read_csv('sub_detect_57/sub_detect_57_0730_lb05849.csv')
# weights: 0.45, 0.35, 0.2

In [107]:
!kaggle competitions submit -c open-images-2019-object-detection -f ensemble_detect_57_0820_1_top100.csv -m "weights: 0.5, 0.3, 0.2"

100%|████████████████████████████████████████| 572M/572M [00:29<00:00, 20.4MB/s]
Successfully submitted to Open Images 2019 - Object Detection

In [35]:
df_sub.PredictionString = pred_strings_top50
df_sub.to_csv('ensemble_detect_57_0808_1_top50.csv', columns=['ImageId', 'PredictionString'], index=False)

In [36]:
!kaggle competitions submit -c open-images-2019-object-detection -f ensemble_detect_57_0808_1_top50.csv -m "submit"

100%|████████████████████████████████████████| 267M/267M [00:13<00:00, 20.3MB/s]
Successfully submitted to Open Images 2019 - Object Detection

In [37]:
!kaggle competitions submissions -c open-images-2019-object-detection

fileName                                     date                 description  status    publicScore  privateScore  
-------------------------------------------  -------------------  -----------  --------  -----------  ------------  
ensemble_detect_57_0808_1_top50.csv          2019-08-08 09:56:38  submit       complete  0.06088      None          
ensemble_detect_57_0808_1.csv                2019-08-08 09:54:15  submit       complete  0.06155      None          
sub_detect_57_cas_1024_0805_1.csv            2019-08-05 07:52:30  submit       complete  0.06012      None          
sub_detect_cas_1024_57_0804.csv              2019-08-04 13:37:48  submit       complete  0.06067      None          
sub_detect_0724.csv                          2019-08-03 00:41:03  test         complete  0.05524      None          
sub_detect_57_0730.csv                       2019-08-03 00:39:53  test         complete  0.05849      None          
sub_cas_0_500_0729.csv                       2019-07-28 17:23:42

In [18]:
""" 
Ensembling methods for object detection.
"""

""" 
General Ensemble - find overlapping boxes of the same class and average their positions
while adding their confidences. Can weigh different detectors with different weights.
No real learning here, although the weights and iou_thresh can be optimized.

Input: 
 - dets : List of detections. Each detection is all the output from one detector, and
          should be a list of boxes, where each box should be on the format 
          [box_x, box_y, box_w, box_h, class, confidence] where box_x and box_y 
          are the center coordinates, box_w and box_h are width and height resp.
          The values should be floats, except the class which should be an integer.

 - iou_thresh: Threshold in terms of IOU where two boxes are considered the same, 
               if they also belong to the same class.
               
 - weights: A list of weights, describing how much more some detectors should
            be trusted compared to others. The list should be as long as the
            number of detections. If this is set to None, then all detectors
            will be considered equally reliable. The sum of weights does not
            necessarily have to be 1.

Output:
    A list of boxes, on the same format as the input. Confidences are in range 0-1.
"""
def GeneralEnsemble(dets, iou_thresh = 0.5, weights=None):
    assert(type(iou_thresh) == float)
    
    ndets = len(dets)
    
    if weights is None:
        w = 1/float(ndets)
        weights = [w]*ndets
    else:
        assert(len(weights) == ndets)
        
        s = sum(weights)
        for i in range(0, len(weights)):
            weights[i] /= s

    out = list()
    used = list()
    
    for idet in range(0,ndets):
        det = dets[idet]
        for box in det:
            if box in used:
                continue
                
            used.append(box)
            # Search the other detectors for overlapping box of same class
            found = []
            for iodet in range(0, ndets):
                odet = dets[iodet]
                
                if odet == det:
                    continue
                
                bestbox = None
                bestiou = iou_thresh
                for obox in odet:
                    if not obox in used:
                        # Not already used
                        if box[4] == obox[4]:
                            # Same class
                            iou = computeIOU(box, obox)
                            if iou > bestiou:
                                bestiou = iou
                                bestbox = obox
                                
                if not bestbox is None:
                    w = weights[iodet]
                    found.append((bestbox,w))
                    used.append(bestbox)
                            
            # Now we've gone through all other detectors
            if len(found) == 0:
                new_box = list(box)
                new_box[5] /= ndets
                out.append(new_box)
            else:
                allboxes = [(box, weights[idet])]
                allboxes.extend(found)
                
                xc = 0.0
                yc = 0.0
                bw = 0.0
                bh = 0.0
                conf = 0.0
                
                wsum = 0.0
                for bb in allboxes:
                    w = bb[1]
                    wsum += w

                    b = bb[0]
                    xc += w*b[0]
                    yc += w*b[1]
                    bw += w*b[2]
                    bh += w*b[3]
                    conf += w*b[5]
                
                xc /= wsum
                yc /= wsum
                bw /= wsum
                bh /= wsum    

                new_box = [xc, yc, bw, bh, box[4], conf]
                out.append(new_box)
    return out
    
def getCoords(box):
    x1 = float(box[0]) - float(box[2])/2
    x2 = float(box[0]) + float(box[2])/2
    y1 = float(box[1]) - float(box[3])/2
    y2 = float(box[1]) + float(box[3])/2
    return x1, x2, y1, y2
    
def computeIOU(box1, box2):
    x11, x12, y11, y12 = getCoords(box1)
    x21, x22, y21, y22 = getCoords(box2)
    
    x_left   = max(x11, x21)
    y_top    = max(y11, y21)
    x_right  = min(x12, x22)
    y_bottom = min(y12, y22)

    if x_right < x_left or y_bottom < y_top:
        return 0.0    
        
    intersect_area = (x_right - x_left) * (y_bottom - y_top)
    box1_area = (x12 - x11) * (y12 - y11)
    box2_area = (x22 - x21) * (y22 - y21)        
    
    iou = intersect_area / (box1_area + box2_area - intersect_area + 1e-6)
    return iou
    
if __name__=="__main__":
    # Toy example
    dets = [ 
            [[0.1, 0.1, 1.0, 1.0, 1, 0.9], [1.2, 1.4, 0.5, 1.5, 1, 0.9]],
            [[0.2, 0.1, 0.9, 1.1, 1, 0.8],[1.19, 1.4, 0.5, 1.5, 1, 0.9]],
            [[5.0,5.0,1.0,1.0,1,0.5]]
           ]
    
    ens = GeneralEnsemble(dets, weights = [1.0, 0.1, 0.5])
    print(ens)

[[0.10909090909090909, 0.1, 0.990909090909091, 1.009090909090909, 1, 0.6125], [1.199090909090909, 1.4000000000000001, 0.5, 1.5, 1, 0.61875], [5.0, 5.0, 1.0, 1.0, 1, 0.16666666666666666]]


In [ ]:
[[0.10909090909090909, 0.1, 0.990909090909091, 1.009090909090909, 'a', 0.6125], [1.2, 1.4, 0.5, 1.5, 'a', 0.3], [5.0, 5.0, 1.0, 1.0, 'a', 0.16666666666666666]]